In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Please switch on the TPU before running these lines.

!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
#huggingface libraries 
import transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset, TensorDataset

#random libraries
from tqdm import tqdm 
import pandas as pd
import numpy as np
import os
import gc
import random
from datasets import load_dataset

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

In [ ]:
train_data=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
test_data=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
train_data

In [ ]:
!pip install datasets

In [ ]:
mnli = load_dataset('glue', 'mnli')
print(len(mnli))
print(mnli['train'])

In [ ]:
mnli_df=pd.DataFrame()
mnli_df['id']=mnli['train']['idx']
mnli_df['premise']=mnli['train']['premise']
mnli_df['hypothesis']=mnli['train']['hypothesis']
mnli_df['lang_abv']=['en']*len(mnli['train']['idx'])
mnli_df['language']=['English']*len(mnli['train']['idx'])
mnli_df['label']=mnli['train']['label']
mnli_df

In [ ]:
# xnli = load_dataset ('xnli', 'all_languages')
# # xnli   = load_dataset ('xnli')
# print(xnli['train'])


In [ ]:
# xnli_df=pd.DataFrame()
# xnli_df['premise']=mnli['train']['premise']
# xnli_df['hypothesis']=mnli['train']['hypothesis']
# xnli_df['label']=mnli['train']['label']

In [ ]:
train_data1=train_data[['premise','hypothesis','label']]
val_data=train_data1[:len(train_data1)//2]
train_data1=train_data1[len(train_data1)//2:]
mnli_df=mnli_df[['premise','hypothesis','label']]
frames=[train_data1,mnli_df]
train_data1=pd.concat(frames)

In [ ]:
print(train_data1)
print(len(val_data))

In [ ]:
print(train_data['label'].shape)

# Preprocessing data

In [ ]:
#model_name = 'bert-base-multilingual-uncased'
#model_name = 'xlm-roberta-base'
model_name = 'joeddav/xlm-roberta-large-xnli' 
batch_size = 32
MAX_LENGTH = 256
NUM_EPOCHS = 3
L_RATE = 1e-5
NUM_CORES = os.cpu_count()

NUM_CORES

In [ ]:
device = xm.xla_device()

print(device)

In [ ]:
#tokenizer=BertTokenizer.from_pretrained(model_name, do_lower_case=True)
tokenizer=XLMRobertaTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
# sample_data = list(zip(train_data['premise'][:2], train_data['hypothesis'][:2]))
# print(sample_data)
# tokenized_sample = tokenizer.batch_encode_plus(sample_data, **kwargs)
# print(tokenized_sample)
def preprocess(data1, tokenizer):
    kwargs = { 'truncation': True,
    'max_length': MAX_LENGTH,
    'padding': 'max_length',
     'return_attention_mask': True, 
    'return_token_type_ids': True     
    }
    data = list(zip(data1['premise'], data1['hypothesis']))
    tokenized = tokenizer.batch_encode_plus(data,**kwargs)
    input_ids = torch.LongTensor(tokenized.input_ids)
    attention_masks = torch.LongTensor(tokenized.attention_mask)
    token_type_ids = torch.LongTensor(tokenized.token_type_ids)
    return input_ids, attention_masks, token_type_ids

In [ ]:
print(train_data1['label'].shape)
#labels=np. reshape(train_data1['label'], (791464,1))
labels = torch.Tensor(train_data1['label']).reshape(-1, 1)
print(labels)

In [ ]:
input_ids, attention_masks, token_type_ids = preprocess(train_data1,tokenizer)
labels = torch.Tensor(train_data1['label']).reshape(-1, 1)
train_dataset_final = TensorDataset(input_ids, attention_masks, token_type_ids,labels)
train_dataloader = DataLoader(train_dataset_final, sampler=RandomSampler(train_dataset_final), batch_size=batch_size)
train_dataset_final

In [ ]:
input_ids1, attention_masks1, token_type_ids1 = preprocess(val_data,tokenizer)
labels1 = torch.Tensor(val_data['label']).reshape(-1, 1)
val_dataset_final = TensorDataset(input_ids1, attention_masks1, token_type_ids1,labels1)
val_dataloader = DataLoader(val_dataset_final, sampler=RandomSampler(val_dataset_final), batch_size=batch_size)
len(val_dataloader)

In [ ]:
print(len(train_dataloader))

In [ ]:
input_ids_test, attention_masks_test, token_type_ids_test = preprocess(test_data,tokenizer)
test_dataset_final = TensorDataset(input_ids_test, attention_masks_test, token_type_ids_test)
test_dataloader = DataLoader(test_dataset_final, sampler=SequentialSampler(test_dataset_final), batch_size=batch_size)
len(test_dataloader)

# Model 

In [ ]:
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3, output_hidden_states=False, output_attentions=False)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

# testing model 

In [ ]:
batch = next(iter(train_dataloader))
b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_token_type_ids = batch[2].to(device)
b_labels = batch[3].to(device)

In [ ]:
outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask,
                labels=b_labels)


In [ ]:
#print(outputs)

In [ ]:
print(outputs[0])
print(outputs[0].item())

# Training model

In [ ]:
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8
            )

In [ ]:
gc.collect()

In [ ]:
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
for epoch in range(NUM_EPOCHS):
    model.train()
    torch.set_grad_enabled(True)
    total_train_loss=0
    epoch_acc_scores_list = []
    for i,batch in tqdm(enumerate(train_dataloader)):
        model.zero_grad()
        input_ids, attention_masks, token_type_ids, labels=batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_masks, labels=labels)
        loss=outputs[0]
        if i%10==0:
            print(f'loss of batch {i}: {loss}')
        total_train_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        xm.optimizer_step(optimizer, barrier=True) 
    print(f'total loss of epoch {epoch}: {total_train_loss}')
    gc.collect()
    
    model.eval()
    torch.set_grad_enabled(False)
    total_val_loss = 0
    targets_list=[]
    for j, val_batch in enumerate(val_dataloader):
        b_input_ids = val_batch[0].to(device)
        b_input_mask = val_batch[1].to(device)
        b_token_type_ids = val_batch[2].to(device)
        b_labels = val_batch[3].to(device)      
        outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        loss = outputs[0]
        total_val_loss = total_val_loss + loss.item()
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0:  # first batch
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    val_acc = accuracy_score(y_true, y_pred)
    print('val accuracy: ',val_acc)


In [ ]:
stacked_val_labels = []
model.eval()

torch.set_grad_enabled(False)
total_val_loss = 0

for j, h_batch in enumerate(test_dataloader):

    b_input_ids = h_batch[0].to(device)
    b_input_mask = h_batch[1].to(device)
    b_token_type_ids = h_batch[2].to(device)     
    outputs = model(b_input_ids, 
            token_type_ids=b_token_type_ids, 
            attention_mask=b_input_mask)
    preds = outputs[0]
    val_preds = preds.detach().cpu().numpy()
    if j == 0:  # first batch
        stacked_val_preds = val_preds

    else:
        stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
        #stacked_val_preds.extend(val_preds)
    #print(len(stacked_val_preds))
    
            
print('\nPrediction complete.')


In [ ]:
print(stacked_val_preds)

In [ ]:
test_preds = np.argmax(stacked_val_preds, axis=1)

In [ ]:
path = '../input/contradictory-my-dear-watson/sample_submission.csv'

df_sample = pd.read_csv(path)

print(df_sample.shape)
df_sample['prediction'] = test_preds

df_sample.head()
df_sample.to_csv('submission.csv', index=False)

# extra code

In [ ]:
# def dataset_lang(data,lang_abv,language):
#     mnli_df=pd.DataFrame()
#     mnli_df['id']=[i for i in range(len(data['train']['premise']))]
#     mnli_df['premise']=mnli['train']['premise']
#     mnli_df['hypothesis']=mnli['train']['hypothesis']
#     mnli_df['lang_abv']=[lang_abv]*len(data['train']['premise'])
#     mnli_df['language']=[language]*len(data['train']['premise'])
#     mnli_df['label']=mnli['train']['label']
#     return mnli_df
# lang_dict={}
# lang_abv=train_data['lang_abv'].unique()
# language=train_data['language'].unique()
# print(lang_abv)
# for i in range(len(lang_abv)):
#     lang_dict[lang_abv[i]]=language[i]
# xnli_df=pd.DataFrame()  
# ind=1
# for i in tqdm(lang_dict):
#     abv=i
#     lang=lang_dict[i]
#     data1=load_dataset ('xnli', abv)
#     get_df=dataset_lang(data1,abv,lang)
#     if ind==1: 
#         xnli_df=get_df
#         ind+=1
#     else:
#         xnli_df=pd.concat([xnli_df,get_df])
# print(xnli_df)        
    